In [13]:
!uv add PyPDF2 --native-tls

Resolved 184 packages in 167ms
Audited 150 packages in 0.05ms


In [32]:
import PyPDF2
import re
with open("/Users/ragulasaikumar/Downloads/crime-and-punishment.pdf",'rb') as pdf_file:
    pdf_reader=PyPDF2.PdfReader(pdf_file)
    text=""
    page_num=0
    for page in pdf_reader.pages:
        if page_num>=6 and page_num<=10:
            curr_page_text=page.extract_text()
            text+=curr_page_text
        page_num+=1
heading_pattern='\d* Free eBooks at Planet eBook\.com|Crime and Punishment\s*\d*'
text=re.sub(heading_pattern,'',text)
text=re.sub('Chapter [A-Z]{1,3}','',text)
text=text.replace(' -\n','').replace("\n","").replace('\x18','')
text=text.lower()

In [33]:
import spacy
preprocess_pipeline=spacy.load('en_core_web_sm')

In [34]:
doc=preprocess_pipeline(text)

In [35]:
vocab={'<UNKNOWN>':0}
dataset=map(lambda sent:list(sent),list(doc.sents))
tokenized_sentences=list(dataset)
io_pairs=[]
for tokens in tokenized_sentences:
    for i in range(1,len(tokens)):
        io_pairs.append(tokens[:i+1])

for i,datapoint in enumerate(io_pairs):
    for j,token in enumerate(datapoint):
        token=str(token)
        if vocab.get(token)==None:
            vocab[token]=len(vocab)
        io_pairs[i][j]=vocab[token]

In [36]:
import torch
torch.tensor(io_pairs[1])[-1]

tensor(3)

In [37]:
from torch.utils.data import Dataset,DataLoader

In [38]:
class BookDataSet(Dataset):
    def __init__(self,dataset):
        self.data=dataset
    def __getitem__(self,idx):
        X=torch.tensor(self.data[idx])[:-1]
        Y=torch.tensor(self.data[idx])[-1]
        return X,Y
    def __len__(self):
        return len(self.data)

dataloader=DataLoader(dataset=BookDataSet(io_pairs),batch_size=1,shuffle=True)

In [39]:
from torch import nn

class NextWordPredictor(nn.Module):
    def __init__(self,vocab_size,hidden_layer_neurons):
        super().__init__()
        self.embedding_layer=nn.Embedding(vocab_size,hidden_layer_neurons)
        self.lstm=nn.LSTM(hidden_layer_neurons,hidden_layer_neurons,batch_first=True)
        self.fc=nn.Linear(hidden_layer_neurons,vocab_size)
    def forward(self,_input):
        embeddings=self.embedding_layer(_input)
        intermediate_hidden_states, (final_hidden_state, final_cell_state)=self.lstm(embeddings)
        return self.fc(final_hidden_state)

In [40]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss


device = torch.device("cpu")
# if torch.backends.mps.is_available() else torch.device("cpu")
predictor=NextWordPredictor(len(vocab),64).to(device)
criterion=CrossEntropyLoss()
optimizer=AdamW(params=predictor.parameters(),lr=0.001)

In [41]:
epochs=50
for epoch in range(epochs):
    for features,output in dataloader:
        features=features.to(device)
        output=output.to(device)
        y_pred=predictor(features)
        loss=criterion(y_pred.squeeze(1),output)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Loss : {loss}")

Loss : 4.557296276092529
Loss : 6.731215000152588
Loss : 7.513824462890625
Loss : 1.3990260362625122
Loss : 2.386157751083374
Loss : 0.039390306919813156
Loss : 5.336611747741699
Loss : 1.0489188432693481
Loss : 3.49109148979187
Loss : 0.8178143501281738
Loss : 1.5251140594482422
Loss : 0.7690956592559814
Loss : 0.07181935012340546
Loss : 0.16466552019119263
Loss : 0.22970281541347504
Loss : 2.9346799850463867
Loss : 0.08589158207178116
Loss : 0.04339710623025894
Loss : 0.1583336889743805
Loss : 0.08999300748109818
Loss : 0.025081543251872063
Loss : 0.05106295645236969
Loss : 0.020524965599179268
Loss : 0.01177067682147026
Loss : 0.08231039345264435
Loss : 0.030689697712659836
Loss : 0.012684616260230541
Loss : 0.007025657221674919
Loss : 0.0033400245010852814
Loss : 0.011088653467595577
Loss : 0.007412785664200783
Loss : 0.010228240862488747
Loss : 0.004220032598823309
Loss : 0.024098344147205353
Loss : 0.005129747558385134
Loss : 0.006055105477571487
Loss : 0.017625020816922188
Loss 

In [75]:
text="It would be".lower()


In [76]:
predictor.eval()
for i in range(30):
    tokens=list(map(lambda text_tok:vocab[text_tok],text.split(" ")))
    next_token=predictor(torch.tensor(tokens))
    token_ind=torch.argmax(next_token)
    text_token=list(vocab.keys())[token_ind]
    text+=f" {text_token}"
    print(text)

it would be interesting
it would be interesting to
it would be interesting to know
it would be interesting to know what
it would be interesting to know what it
it would be interesting to know what it is
it would be interesting to know what it is men
it would be interesting to know what it is men are
it would be interesting to know what it is men are most
it would be interesting to know what it is men are most afraid
it would be interesting to know what it is men are most afraid of
it would be interesting to know what it is men are most afraid of .
it would be interesting to know what it is men are most afraid of . taking
it would be interesting to know what it is men are most afraid of . taking a
it would be interesting to know what it is men are most afraid of . taking a new
it would be interesting to know what it is men are most afraid of . taking a new step
it would be interesting to know what it is men are most afraid of . taking a new step ,
it would be interesting to know what it

'to'